In [1]:
import csv
import requests
from bs4 import BeautifulSoup
from datetime import date
import os
import re
from svgimgutils import SVGImgUtils
import json
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from svgpathtools import parse_path, Line, Path, wsvg
from random import randint
from os import listdir
from os.path import isfile, join
import cairosvg

In [193]:
# Make file paths
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(os.path.abspath(''))))
json_data_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/diplomacy_gif_maker/json_data')
csv_data_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/diplomacy_gif_maker/csv')
images_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/diplomacy_gif_maker/images')
svg_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/diplomacy_gif_maker/images/svg')
png_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/diplomacy_gif_maker/images/png')
gif_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/diplomacy_gif_maker/images/gif/game-modified3.gif')
print(json_data_path)
print(png_path)
print(svg_path)
print(gif_path)

/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/json_data
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/svg
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/gif/game-modified3.gif


In [148]:
def game_url_builder(game_name: str, game_code: str, current_year: int, current_season: str):
    backstabbr_url = 'https://www.backstabbr.com/game/'
    first_year = 1901
    
    base_url = backstabbr_url + game_name + '/' + game_code + '/'
    seasons = ['spring', 'fall', 'winter']
    season_urls = [str(base_url+'****/spring'), str(base_url+'****/fall'), str(base_url+'****/winter')]
    first_url = season_urls[0].replace("****", str(first_year)) 
    
    url_list = []
    years_list = []
    season_list = []
    
    # Calculate number of urls to build
    number_of_years = current_year - first_year
    base_number = number_of_years*3
    if current_season=="spring":
        number_of_urls = base_number + 1
    elif current_season=="fall":
        number_of_urls = base_number + 2
    else:
        number_of_urls = base_number + 3
    if number_of_years==0:
        years_list.append('1901')
    else:
        for num in range(0,number_of_years):
            year = first_year+num
            years_list.append(str(year))
    current_url_number = 1
    for i, current_year in enumerate(years_list):
        if current_url_number == number_of_urls:
            break
        year_url_list = []
        for j, item in enumerate(season_urls):
            season_list.append(seasons[j])
            if current_url_number == number_of_urls:
                break
            year_url_list.append(item.replace("****", str(current_year)))
            current_url_number+=1
        for url in year_url_list:
            url_list.append(url) 
    url_list.append(base_url)
    return url_list, years_list, season_list  

In [149]:
# War on drugs game
game_name = 'The-war-on-drugs'
game_code = '5704487623196672'

In [150]:
urls, years, seasons = game_url_builder(game_name, game_code, 1921, 'fall')

In [151]:
urls[0][66:70]

'1901'

In [6]:
# Get all orders data for the game
def get_script_data(script_string, beginning, end):
        start = script_string.find(beginning) + len(beginning)
        end = script_string.find(end)
        substring = script_string[start:end]
        return str(substring)

for i, url in enumerate(urls):
    # Get game data here
    page = requests.get(url).content
    soup = BeautifulSoup(page)
    # Find all script tags in the page
    all_scripts = soup.find_all('script')
    # Desired order json is the 10th script. Order does not change, unless there are changes to the Backstabbr code!
    javascript_script_target = all_scripts[10]
    javascript_string = str(javascript_script_target)
    orders_string = get_script_data(javascript_string, "var orders = ", ";\n  var unitsByPlayer")
    orders_json = json.loads(orders_string)
    unitsByPlayer_string = get_script_data(javascript_string, 'var unitsByPlayer = ', ";\n  var territories =")
    unitsByPlayer_json = json.loads(unitsByPlayer_string)
    territories_string = get_script_data(javascript_string, 'var territories = ', ";\n  var mapSize")
    territories_json = json.loads(territories_string)
    stage_string = get_script_data(javascript_string, 'var stage = ', ';\n  var orders')
    stage_json = json.loads(stage_string)
    unitChangeCount_string = get_script_data(javascript_string, 'var unitChangeCount = ', ';\n  var buildableTerritories =')
    unitChangeCount_json = json.loads(unitChangeCount_string)
    json_list = [orders_json, unitsByPlayer_json, territories_json, stage_json, unitChangeCount_json]
    json_filename = ['orders', 'unitsByPlayer', 'territories', 'stage', 'unitChangeCount']
    for i, item in enumerate(json_list):
        with open(json_data_path + '/' + url[66:70] + '_' + url[71:] + '_' + str(json_filename[i]) + '_' + '.txt', 'w') as outfile:
            json.dump(item, outfile)
    sleep(randint(3,10))

NameError: name 'json_data_path' is not defined

In [ ]:
# Get svg image of units and borders from all urls
for i, url in enumerate(urls):
    sleep(randint(3,10))
    driver = webdriver.Firefox()
    driver.get(url)
    xpath = "//*[name()='svg']//*[name()='path']";
    target_id = 'map'
    elem = driver.find_element_by_id(target_id)
    html = elem.get_attribute('innerHTML')
    driver.close()
    
    # Parse svg
    soup = BeautifulSoup(html,'lxml')
    soup.image.decompose()
    paths = soup.find_all('svg')
    paths_string = str(paths)
    svg_file_text = '<?xml version="1.0" encoding="utf-8" ?>' + paths_string[1:-1]
    svg_file_text = svg_file_text.replace("viewbox", "viewBox")
    svg_file_text = svg_file_text.replace('<image height="560" preserveaspectratio="none" transform="matrix(1,0,0,1,0,0)" width="610" x="-1" xlink:href="../map_background.png" y="-1"/>', '')
    svg_file_text = svg_file_text.replace('style="overflow: hidden; position: relative;"', 'style="overflow: hidden; position: relative; background-color: white"')
    # Save svg
    text_file = open("images/svg/season"+str(i)+".svg", "w")
    n = text_file.write(svg_file_text)
    text_file.close()

In [15]:
# Convert SVGs to PNGs
svg_files = [f for f in listdir(svg_path) if isfile(join(svg_path, f))]
for item in range(0,len(svg_files)-1):
    cairosvg.svg2png(
        url=str(svg_path)+"/season"+str(item)+".svg", write_to=str(png_path)+"/season"+str(item)+".png")

In [52]:
# import svgwrite
# dwg = svgwrite.Drawing('test.svg', profile='tiny')
# dwg.add(dwg.line((0, 0), (10, 0), stroke=svgwrite.rgb(10, 10, 16, '%')))
# dwg.add(dwg.text('Test', insert=(0, 0.2)))
# dwg.save()

In [11]:
images = []
filenames = [f for f in listdir(png_path) if isfile(join(png_path, f))]
sub = 'modified'
filenames_list = [i for i in filenames if sub not in i]
# Human sorting of file name list
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

filenames_list.sort(key=natural_keys)
filenames_list = filenames_list[1:]
print(filenames_list)

['season18.png',
 'season50.png',
 'season56.png',
 'season48.png',
 'season2.png',
 'season35.png',
 'season15.png',
 'season19.png',
 'season59.png',
 'season58.png',
 'season16.png',
 'season27.png',
 'season49.png',
 'season21.png',
 'season13.png',
 'season6.png',
 'season14.png',
 'season26.png',
 'season57.png',
 'season34.png',
 'season40.png',
 'season31.png',
 'season52.png',
 'season32.png',
 'season55.png',
 'season39.png',
 'season45.png',
 'season4.png',
 'season38.png',
 'season8.png',
 'season5.png',
 'season24.png',
 'season44.png',
 'season53.png',
 'season28.png',
 'season3.png',
 'season10.png',
 'season42.png',
 'season51.png',
 'season54.png',
 'season46.png',
 'season30.png',
 'season1.png',
 'season41.png',
 'season9.png',
 'season43.png',
 'season11.png',
 'season20.png',
 'season22.png',
 'season7.png',
 'season33.png',
 'season37.png',
 'season23.png',
 'season36.png',
 'season17.png',
 'season29.png',
 'season25.png',
 'season47.png',
 'season0.png',
 'seaso

In [18]:
from PIL import Image
filenames = [f for f in listdir(png_path) if isfile(join(png_path, f))]
for i, file in enumerate(filenames_list):
    image = Image.open(str(png_path)+'/season' + str(i) + '.png')
    image.convert("RGBA") # Convert this to RGBA if possible
    canvas_grey_value = 180
    alpha = 255
    canvas = Image.new('RGBA', image.size, (canvas_grey_value,canvas_grey_value,canvas_grey_value,alpha)) # Empty canvas colour (r,g,b,a)
    canvas.paste(image, mask=image) # Paste the image onto the canvas, using it's alpha channel as mask
    canvas.save(str(png_path)+'/modified-season' + str(i) + '.png', format="PNG")

In [19]:
# Make GIF
gif_path = os.path.join(BASE_DIR, 'diplomacy_gif_maker/diplomacy_gif_maker/images/gif/game-modified3.gif')

gif_filename_list = []
for i, file in enumerate(filenames_list):
    file_string = png_path + '/modified-season' + str(i) + '.png'
    gif_filename_list.append(file_string)
#     print(gif_filename_list)
images = []
for filename in gif_filename_list:
    images.append(imageio.imread(filename))
imageio.mimsave(gif_path, images, fps=1.5)

/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png/modified-season0.png
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png/modified-season1.png
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png/modified-season2.png
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png/modified-season3.png
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png/modified-season4.png
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png/modified-season5.png
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png/modified-season6.png
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png/modified-season7.png
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png/modified-season8.png
/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images/png/modified-season9.png


In [18]:
images_path

'/home/bking/Projects/poetry/diplomacy_gif_maker/diplomacy_gif_maker/images'

In [21]:
# Create SVG Image Utils for each SVG image
base_template = SVGImgUtils.fromfile(images_path + '/map_background_real.svg')
action = SVGImgUtils.fromfile(svg_path + '/season0.svg')
base_template.append(action)
base_template.save(images_path + '/merged.svg')

IndexError: list index out of range

In [14]:
import svgutils.transform as st
template = st.fromfile(images_path + '/map_background_real_scaled.svg')
second_svg = st.fromfile(svg_path + '/season0-scaled.svg')
template.append(second_svg)
template.save(images_path + '/merged2.svg')

Wrangling JSON data to pandas dataframe

In [64]:
# Define test url
test_url = urls[3]
print(test_url)

https://www.backstabbr.com/game/The-war-on-drugs/5704487623196672/1902/spring


In [146]:
filetypes = ['orders', 'stage', 'territories', 'unitChangeCount', 'unitsByPlayer']

In [181]:
# Wrangle units data into dataframe
def getUnitsByPlayer(unitsByPlayerData, year, season):
    country_list = []
    territory_list = []
    unit_type_list = []
    years_list = []
    seasons_list = []
    for player in unitsByPlayerData.items():
        units = player[1]
        for territory, unit_type in units.items():
            country_list.append(player[0])
            territory_list.append(territory)
            unit_type_list.append(unit_type)
            years_list.append(year)
            seasons_list.append(season)
    data_tuples = list(zip(country_list, territory_list, unit_type_list, years_list, seasons_list))
    df = pd.DataFrame(data_tuples, columns=['country','territory', 'unit_type', 'year', 'season'])
    return df

# Wrangle UnitChangeCounts into dataframe
def getUnitChangeCounts(unitChangeCountData, year, season='winter'):
    country_list = []
    unitChangeCount_list = []
    years_list = []
    seasons_list = []
    for player in unitChangeCountData.items():
        country_list.append(player[0])
        unitChangeCount_list.append(player[1])
        years_list.append(year)
        seasons_list.append(season)
    data_tuples = list(zip(country_list, unitChangeCount_list, years_list, seasons_list))
    df = pd.DataFrame(data_tuples, columns=['country','unitChange', 'year', 'season'])
    return df

# Wrangle orders data into dataframe
def getOrders(orderData, year, season):
    country_list = []
    type_list = []
    unitFrom_list = []
    unitTo_list = []
    result_list = []
    result_reason_list = []
    years_list = []
    seasons_list = []
    for player in data.items():
        for territory in player[1].items():
            country_list.append(player[0])
            unitFrom_list.append(territory[0])
            for to_order in territory[1].items():
                if to_order[0] == 'to':
                    unitTo_list.append(to_order[1])
                elif to_order[0] == 'type':
                    type_list.append(to_order[1])
                elif to_order[0] == 'result':
                    result_list.append(to_order[1])
                elif to_order[0] == 'result_reason':
                    result_reason_list.append(to_order[1])
                years_list.append(year)
                seasons_list.append(season)
    data_tuples = list(zip(country_list, type_list, unitFrom_list, unitTo_list, result_list, result_reason_list, years_list, seasons_list))
    df = pd.DataFrame(data_tuples, columns=['country', 'type', 'unitFrom_list','unitTo_list','result_list', 'result_reason_list','year', 'season'])    
    return df

In [182]:
years[0]

'1901'

In [192]:
filetypes = ['orders', 'stage', 'territories', 'unitChangeCount', 'unitsByPlayer']
year_index = -1
orders_data_combined = []
UnitsByPlayer_data_combined = []
unitChangeCount_data_combined = []
for file_type in filetypes:
    with open(json_data_path + '/1901_spring_'+str(file_type)+'_.txt') as json_file:
        data = json.load(json_file)
    if file_type=='orders':
        orders_data_combined = getOrders(data, '1901', 'spring')
    elif file_type=='unitChangeCount':
        unitChangeCount_data_combined = getUnitChangeCounts(data, '1901', 'spring')
    elif file_type=='unitsByPlayer':
        UnitsByPlayer_data_combined = getUnitsByPlayer(data, '1901', 'spring')
    else:
        continue

for i, season in enumerate(seasons[1:]):
    if i%3==0:
        year_index+=1
    for file_type in filetypes:
        with open(json_data_path + '/' + years[year_index] +'_'+str(season)+'_'+str(file_type)+'_.txt') as json_file:
            data = json.load(json_file)
        if file_type == 'orders':
#             print("Appending orders")
            orders_data = getOrders(data, str(years[year_index]), season)
            orders_data_combined = orders_data_combined.append(orders_data, ignore_index = True)
#             print(orders_data_combined)
        elif file_type == 'unitsByPlayer':
            UnitsByPlayer_data = getUnitsByPlayer(data, str(years[year_index]), season)
            UnitsByPlayer_data_combined = UnitsByPlayer_data_combined.append(UnitsByPlayer_data, ignore_index = True)
#             print(UnitsByPlayer_data_combined)
        elif file_type == 'unitChangeCount':
            unitChangeCount_data = getUnitChangeCounts(data, str(years[year_index]), season)
            unitChangeCount_data_combined = unitChangeCount_data_combined.append(unitChangeCount_data, ignore_index = True)
        else:
            continue 
full_data_list = [orders_data_combined, UnitsByPlayer_data_combined, unitChangeCount_data_combined]
full_datanames_list = ['orders', 'unitsByPlayer', 'unitChangeCount']

In [194]:
for i, dataset in enumerate(full_data_list):
    dataset.to_csv(csv_data_path + '/' + full_datanames_list[i] + '_full.csv')

In [31]:
df = pd.DataFrame.from_dict(data, orient='columns')

In [40]:
pd.json_normalize(data['Turkey'], max_level=1)

,Ank,Bul,Smy
0,F,A,A


In [20]:
for item in orders_json:
    print(item)

Turkey
Italy
France
Austria
Germany
England
Russia


In [21]:
territories_json

{'Par': 'France',
 'Smy': 'Turkey',
 'Lon': 'England',
 'Nap': 'Italy',
 'Ven': 'Italy',
 'Mun': 'Germany',
 'War': 'Russia',
 'Mar': 'France',
 'Mos': 'Russia',
 'Kie': 'Germany',
 'Tri': 'Austria',
 'Lvp': 'England',
 'Stp': 'Russia',
 'Ber': 'Germany',
 'Con': 'Turkey',
 'Sev': 'Russia',
 'Rom': 'Italy',
 'Vie': 'Austria',
 'Edi': 'England',
 'Ank': 'Turkey',
 'Bud': 'Austria',
 'Bre': 'France'}

In [28]:
pd.DataFrame.from_records(orders_json, index=range(0,len(orders_json)))

,Austria,England,France,Germany,Italy,Russia,Turkey
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
train = pd.DataFrame.from_dict(orders_json, orient='index')
train.reset_index(level=0, inplace=True)

In [30]:
train

,index,Ank,Smy,Con,Ven,Nap,Rom,Par,Mar,Bre,...,Ber,Mun,Kie,Edi,Lon,Lvp,Mos,Sev,Stp,War
0,Turkey,"{'to': 'BLA', 'type': 'MOVE', 'result': 'FAILS...","{'to': 'Lon', 'type': 'MOVE', 'result': 'FAILS...","{'to': 'Bul', 'type': 'MOVE', 'result': 'SUCCE...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Italy,NaN,NaN,NaN,"{'type': 'HOLD', 'result': 'SUCCEEDS', 'result...","{'to': 'ION', 'type': 'MOVE', 'result': 'SUCCE...","{'to': 'Apu', 'type': 'MOVE', 'result': 'SUCCE...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,France,NaN,NaN,NaN,NaN,NaN,NaN,"{'to': 'Bur', 'type': 'MOVE', 'result': 'SUCCE...","{'to': 'Spa', 'type': 'MOVE', 'result': 'SUCCE...","{'to': 'MAO', 'type': 'MOVE', 'result': 'SUCCE...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"{'to': 'Kie', 'type': 'MOVE', 'result': 'SUCCE...","{'to': 'Ruh', 'type': 'MOVE', 'result': 'SUCCE...","{'to': 'Den', 'type': 'MOVE', 'result': 'SUCCE...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,England,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,"{'to': 'NWG', 'type': 'MOVE', 'result': 'SUCCE...","{'to': 'NTH', 'type': 'MOVE', 'result': 'SUCCE...","{'to': 'Edi', 'type': 'MOVE', 'result': 'SUCCE...",NaN,NaN,NaN,NaN
6,Russia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'to': 'Stp', 'type': 'MOVE', 'result': 'SUCCE...","{'to': 'BLA', 'type': 'MOVE', 'result': 'FAILS...","{'to': 'BOT', 'type': 'MOVE', 'result': 'SUCCE...","{'type': 'HOLD', 'result': 'SUCCEEDS', 'result..."
